<img width="200px" align="left" style="margin-right:20px" src="https://static.tildacdn.com/tild3038-6565-4361-a462-366365643138/new_logo.png"> <br /><br /><br /><br /><b>New Professions Lab</b> <br /> Специалист по большим данным

# Проект 1

# Спрогнозировать пол и возрастную категорию интернет-пользователей по логу посещения сайтов

<img width="110px" align="left" src="http://data.cluster-lab.com/public-newprolab-com/project01_img0.png?img">

Одна из задач DMP-системы состоит в том, чтобы по разрозненным даннным, таким, как посещения неким пользователем сайтов, классифицировать его и присвоить ему определённую категорию: пол, возраст, интересы и так далее. В дальнейшем составляется портрет, или профиль, пользователя, на основе которого ему более таргетированно показывается реклама в интернете.

### Задача

Используя доступный набор данных о посещении страниц у одной части пользователей, сделать прогноз относительно **пола и возрастной категории** другой части пользователей. Угадывание (hit) - правильное предсказание и пола, и возрастной категории одновременно.

Мы не ограничиваем вас в выборе инструментов и методов работы с данными. Используйте любые эвристики, внешние источники, парсинг контента страниц — всё, что поможет вам выполнить задачу. Единственное ограничение — никаких ручных действий. Руками проставлять классы нельзя.

Поскольку это ваш проект, который мы наверняка захотите показать другим, уделите его оформлению достаточно времени. Мы рекомендуем сделать весь проект в этом ноутбуке. Снизу, под заданием, вы сможете описать ваше решение.

⏰ **Дедлайн: 8 ноября 2020, 23:59**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os, sys
import json
import pickle
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote
from pandas.io.json import json_normalize
import datetime
from sklearn import preprocessing

### Обработка данных на вход

In [2]:
file_path = '/data/share/project01/gender_age_dataset.txt'

Файл содержит данные о посещении сайтов ~40 000 пользователей, при этом по некоторым из них (~ 35 000) известны их пол и возрастная категория, а по 5 000 - эта информация не известна. В файле есть 4 поля:
* **gender** - пол, принимающий значения `M` (male - мужчина), `F` (female - женщина), `-` (пол неизвестен);
* **age** - возраст, представленный в виде диапазона x-y (строковый тип), или `-` (возрастная категория неизвестна);
* **uid** - идентификатор пользователя, строковая переменная;
* **user_json** - поле json, в котором содержатся записи о посещении сайтов этим пользователем `(url, timestamp)`.

Первое, что обычно делают в таких случаях — исследуют имеющийся датасет и разбираются, какие же данные мы получили.

Загрузим весь датасет в pandas:

In [3]:
df = pd.read_csv(file_path, sep='\t')

И теперь попробуем понять, что у нас есть:

In [4]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


In [5]:
df.shape

(41138, 4)

Убираем верхний уровень дикта и получаем листы диктов в строках

In [6]:
df['user_json'] = json_normalize(df['user_json'].apply(json.loads))
df.shape

(41138, 4)

In [7]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,[{'url': 'http://zebra-zoya.ru/200028-chehol-o...
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"[{'url': 'http://sweetrading.ru/?p=900', 'time..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,[{'url': 'http://ru.oriflame.com/products/prod...
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,[{'url': 'http://translate-tattoo.ru/font-sele...
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"[{'url': 'https://mail.rambler.ru/#/folder/', ..."


Конвертируем датасет в дикт построчно

In [8]:
df2_dict = df.to_dict('records')

Разбиваем дикт на два столбца

In [9]:
df2_norm = pd.DataFrame(json_normalize(df2_dict, 'user_json', ['gender', 'age', 'uid']),columns=['gender', 'age', 'uid','timestamp','url'])

In [10]:
df2_norm.shape

(5829507, 5)

In [11]:
df2_norm.head()

,gender,age,uid,timestamp,url
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,1419688144068,http://zebra-zoya.ru/200028-chehol-organayzer-...
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,1426666298001,http://news.yandex.ru/yandsearch?cl4url=chezas...
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,1426666298000,http://www.sotovik.ru/news/240283-htc-one-m9-z...
3,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,1426661722001,http://news.yandex.ru/yandsearch?cl4url=chezas...
4,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,1426661722000,http://www.sotovik.ru/news/240283-htc-one-m9-z...


Методом `pandas.DataFrame.apply` (хотя не только им) можно применить операцию десериализации json-строк ко всему датасету. Рекомендуем почитать [документацию по методу apply](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

Работая с подобными операциями, обратите внимание на kwargs-аргумент `axis`. Часто, забыв его указать, вы примените операцию не к ряду (строке), а к столбцу, что вряд ли входит в ваши планы.

### Очистка данных и feature engineering

Очистка данных и генерация новых фич составит значительную часть вашей работы. Именно здесь вы и должны продемонстрировать знания и креативность: чем лучше окажутся ваши фичи и чем лучше сможете убрать шум из датасета, тем лучших результатов вы достигнете.

Одна из первых вещей, которые можно попробовать — это вытащить домены и использовать их в качестве признаков. Можно воспользоваться функцией:

In [12]:
def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    if netloc is not None: return str(netloc.encode('utf8')).strip()
    return None

Поскольку эта часть и есть ваша работа, мы не станем раскрывать все секреты (хотя несколько советов мы всё же дали, посмотрите ниже в разделе Подсказки).

Добавляем новый столбец, состоящий из домейнов

In [13]:
df2_norm['domain'] = df2_norm['url'].apply(url2domain)
df2_norm.drop(labels='url',axis=1, inplace=True)
df2_norm.head()

,gender,age,uid,timestamp,domain
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,1419688144068,b'zebra-zoya.ru'
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,1426666298001,b'news.yandex.ru'
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,1426666298000,b'sotovik.ru'
3,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,1426661722001,b'news.yandex.ru'
4,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,1426661722000,b'sotovik.ru'


In [14]:
df2_norm['timestamp'].shape, df2_norm['timestamp'].unique().shape

((5829507,), (4164808,))

In [15]:
df2_norm['domain'].shape, df2_norm['domain'].unique().shape

((5829507,), (116988,))

In [16]:
def datetime_conv(value):
    return datetime.datetime.fromtimestamp(value / 1e3)

In [17]:
df2_norm['datetime'] = df2_norm['timestamp'].apply(datetime_conv)
df2_norm['datetime'] = pd.to_datetime(df2_norm['datetime'], format='%Y%m%d%h%')
df2_norm.drop(labels='timestamp',axis=1, inplace=True)
df2_norm.head()

,gender,age,uid,domain,datetime
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'zebra-zoya.ru',2014-12-27 16:49:04.068
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 11:11:38.001
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 11:11:38.000
3,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 09:55:22.001
4,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 09:55:22.000


In [18]:
df2_norm['age'].unique()

array(['18-24', '25-34', '>=55', '45-54', '35-44', '-'], dtype=object)

In [19]:
df2_norm['gender'].unique()

array(['F', 'M', '-'], dtype=object)

In [20]:
df3 = df2_norm
df3.head()

,gender,age,uid,domain,datetime
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'zebra-zoya.ru',2014-12-27 16:49:04.068
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 11:11:38.001
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 11:11:38.000
3,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 09:55:22.001
4,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 09:55:22.000


In [21]:
df3['hour'] = df3['datetime'].dt.hour
df3['day'] = df3['datetime'].dt.day
df3['month'] = df3['datetime'].dt.month

In [22]:
df3['day_part'] = 'Unknown'
df3.loc[df3['hour'].between(6, 12, inclusive=True), 'day_part'] = 'Morning'
df3.loc[df3['hour'].between(13, 18, inclusive=True), 'day_part'] = 'Afternoon'
df3.loc[df3['hour'].between(19, 24, inclusive=True), 'day_part'] = 'Evening'
df3.loc[df3['hour'].between(0, 5, inclusive=True), 'day_part'] = 'Night'
df3.head()

,gender,age,uid,domain,datetime,hour,day,month,day_part
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'zebra-zoya.ru',2014-12-27 16:49:04.068,16,27,12,Afternoon
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 11:11:38.001,11,18,3,Morning
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 11:11:38.000,11,18,3,Morning
3,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 09:55:22.001,9,18,3,Morning
4,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 09:55:22.000,9,18,3,Morning


In [23]:
df3['season'] = 'Unknown'
df3.loc[df3['month'].between(3, 5, inclusive=True), 'season'] = 'Spring'
df3.loc[df3['month'].between(6, 8, inclusive=True), 'season'] = 'Summer'
df3.loc[df3['month'].between(9, 11, inclusive=True), 'season'] = 'Autumn'
df3.loc[df3['month'].between(1, 2, inclusive=True), 'season'] = 'Winter'
df3.loc[df3['month'].between(12, 12, inclusive=True), 'season'] = 'Winter'
df3.head()

,gender,age,uid,domain,datetime,hour,day,month,day_part,season
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'zebra-zoya.ru',2014-12-27 16:49:04.068,16,27,12,Afternoon,Winter
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 11:11:38.001,11,18,3,Morning,Spring
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 11:11:38.000,11,18,3,Morning,Spring
3,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 09:55:22.001,9,18,3,Morning,Spring
4,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 09:55:22.000,9,18,3,Morning,Spring


In [24]:
df3['week_day'] = df3['datetime'].dt.day_name()
workdays = ['Wednesday', 'Thursday', 'Friday', 'Monday', 'Tuesday']
weekend = ['Saturday', 'Sunday']
df3.head()

,gender,age,uid,domain,datetime,hour,day,month,day_part,season,week_day
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'zebra-zoya.ru',2014-12-27 16:49:04.068,16,27,12,Afternoon,Winter,Saturday
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 11:11:38.001,11,18,3,Morning,Spring,Wednesday
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 11:11:38.000,11,18,3,Morning,Spring,Wednesday
3,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 09:55:22.001,9,18,3,Morning,Spring,Wednesday
4,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 09:55:22.000,9,18,3,Morning,Spring,Wednesday


In [25]:
df3['day_type'] = df3['week_day'].isin(workdays)
df3['day_type'] = df3['day_type'].replace([True, False], ['Workday', 'Weekend'])
df3.head()

,gender,age,uid,domain,datetime,hour,day,month,day_part,season,week_day,day_type
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'zebra-zoya.ru',2014-12-27 16:49:04.068,16,27,12,Afternoon,Winter,Saturday,Weekend
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 11:11:38.001,11,18,3,Morning,Spring,Wednesday,Workday
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 11:11:38.000,11,18,3,Morning,Spring,Wednesday,Workday
3,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 09:55:22.001,9,18,3,Morning,Spring,Wednesday,Workday
4,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 09:55:22.000,9,18,3,Morning,Spring,Wednesday,Workday


In [26]:
df3.head()

,gender,age,uid,domain,datetime,hour,day,month,day_part,season,week_day,day_type
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'zebra-zoya.ru',2014-12-27 16:49:04.068,16,27,12,Afternoon,Winter,Saturday,Weekend
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 11:11:38.001,11,18,3,Morning,Spring,Wednesday,Workday
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 11:11:38.000,11,18,3,Morning,Spring,Wednesday,Workday
3,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2015-03-18 09:55:22.001,9,18,3,Morning,Spring,Wednesday,Workday
4,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2015-03-18 09:55:22.000,9,18,3,Morning,Spring,Wednesday,Workday


In [27]:
df3.shape

(5829507, 12)

In [28]:
df4 = df3.drop_duplicates(subset=['uid','domain','day_part', 'season', 'week_day', 'day_type'], keep='first')
df4['uid'].shape, df4['uid'].unique().shape

((1103925,), (41138,))

In [29]:
df4.drop(labels=['datetime','hour','day','month'],axis=1, inplace=True)
df4.head()

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,gender,age,uid,domain,day_part,season,week_day,day_type
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'zebra-zoya.ru',Afternoon,Winter,Saturday,Weekend
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',Morning,Spring,Wednesday,Workday
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',Morning,Spring,Wednesday,Workday
5,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,b'sweetrading.ru',Night,Winter,Sunday,Weekend
8,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,b'101.ru',Night,Winter,Sunday,Weekend


In [30]:
len(set(df4[~((df4.gender == '-') & (df4.age == '-'))]['domain']))

111581

In [31]:
len(set(df4[((df4.gender == '-') & (df4.age == '-'))]['domain']))

19663

In [32]:
len(list(set(df4[~((df4.gender == '-') & (df4.age == '-'))]['domain']).intersection(set(df4[((df4.gender == '-') & (df4.age == '-'))]['domain']))))

14256

Пересечение по сайтам для тестовой выборки:

In [33]:
14256/19663

0.7250165285053145

Модель уже можно обучать на таких данных, предварительно сделав label encoding по домейнам

In [34]:
le_domain = preprocessing.LabelEncoder()
le_domain.fit(df4['domain'].astype(str)) 

LabelEncoder()

Пробуем обучать модель и смотрим на результат, если не хватает, продолжаем работать с данными

### Деление на train и test сеты, обучение модели, предсказания для test-сета

Давайте теперь оценим размер нашего train и test сетов. Train set:

In [35]:
df4_train = df4[~((df4.gender == '-') & (df4.age == '-'))]
len(df4_train[~((df4_train.gender == '-') & (df4_train.age == '-'))])

1015189

In [36]:
df4_train.head()

,gender,age,uid,domain,day_part,season,week_day,day_type
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'zebra-zoya.ru',Afternoon,Winter,Saturday,Weekend
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',Morning,Spring,Wednesday,Workday
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',Morning,Spring,Wednesday,Workday
5,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,b'sweetrading.ru',Night,Winter,Sunday,Weekend
8,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,b'101.ru',Night,Winter,Sunday,Weekend


Test set:

In [37]:
df4_test = df4[((df4.gender == '-') & (df4.age == '-'))]
len(df4_test[((df4_test.gender == '-') & (df4_test.age == '-'))])

88736

In [38]:
df4_test.head()

,gender,age,uid,domain,day_part,season,week_day,day_type
5312952,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'interfax.ru',Afternoon,Winter,Sunday,Weekend
5312953,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'amerikan-gruzovik.ru',Afternoon,Winter,Saturday,Weekend
5312969,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'tv.jampo.tv',Morning,Winter,Saturday,Weekend
5312970,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'sovsekretno.ru',Morning,Winter,Friday,Workday
5312971,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'marketgid.com',Morning,Winter,Friday,Workday


Теперь делаем encoding по остальным фичам и делим train на train и val

Инициализируем энкодеры для каждой фичи

In [39]:
# features
le_day_part = preprocessing.LabelEncoder()
le_season   = preprocessing.LabelEncoder()
le_week_day = preprocessing.LabelEncoder()
le_day_type = preprocessing.LabelEncoder()

# targets
le_gender = preprocessing.LabelEncoder()
le_age    = preprocessing.LabelEncoder()

Фитим все энкодеры

In [40]:
# features
le_day_part.fit(df4_train['day_part'].astype(str))
le_season  .fit(df4_train['season'].astype(str))
le_week_day.fit(df4_train['week_day'].astype(str))
le_day_type.fit(df4_train['day_type'].astype(str))

# targets
le_gender.fit(df4_train['gender'].astype(str))
le_age   .fit(df4_train['age'].astype(str))

LabelEncoder()

# Продолжение работы с датасетом . . .

Считаем статистику по каждому сайту:

In [296]:
sex_stat = pd.DataFrame(df4_train[['domain','gender']].groupby([pd.Grouper(key='domain'), 'gender']).size(), columns=['sex_stat'])
sex_stat.head(10)

sex_stat
domain            gender          
b'zebra-zoya.ru'  F              3
b'news.yandex.ru' F           1202
                  M           2301
b'sotovik.ru'     F              7
                  M             21
b'sweetrading.ru' M              2
b'101.ru'         F            301
                  M            460
b'oesex.ru'       M              3
b'interfax.ru'    F            588

In [297]:
age_stat = pd.DataFrame(df4_train[['domain','age']].groupby([pd.Grouper(key='domain'), 'age']).size(), columns=['age_stat'])
age_stat.head(10)

age_stat
domain            age            
b'zebra-zoya.ru'  18-24         1
                  35-44         1
                  >=55          1
b'news.yandex.ru' 18-24       310
                  25-34      1370
                  35-44      1002
                  45-54       557
                  >=55        264
b'sotovik.ru'     18-24         2
                  25-34        18

In [298]:
sex_stat = pd.pivot_table(sex_stat, values='sex_stat', index='domain', columns='gender',
                           aggfunc='max').fillna(0)
sex_stat.reset_index(inplace=True)
sex_stat.head()

gender,domain,F,M
0,b'zebra-zoya.ru',3.0,0.0
1,b'news.yandex.ru',1202.0,2301.0
2,b'sotovik.ru',7.0,21.0
3,b'sweetrading.ru',0.0,2.0
4,b'101.ru',301.0,460.0


In [299]:
sex_stat[['F','M']] = sex_stat[['F','M']].div(sex_stat[['F','M']].sum(axis=1), axis=0)
sex_stat.head()

gender,domain,F,M
0,b'zebra-zoya.ru',1.000000,0.000000
1,b'news.yandex.ru',0.343134,0.656866
2,b'sotovik.ru',0.250000,0.750000
3,b'sweetrading.ru',0.000000,1.000000
4,b'101.ru',0.395532,0.604468


In [300]:
age_stat = pd.pivot_table(age_stat, values='age_stat', index='domain', columns='age',
                           aggfunc='max').fillna(0)
age_stat.reset_index(inplace=True)
age_stat.head()

age,domain,18-24,25-34,35-44,45-54,>=55
0,b'zebra-zoya.ru',1.0,0.0,1.0,0.0,1.0
1,b'news.yandex.ru',310.0,1370.0,1002.0,557.0,264.0
2,b'sotovik.ru',2.0,18.0,4.0,4.0,0.0
3,b'sweetrading.ru',0.0,2.0,0.0,0.0,0.0
4,b'101.ru',65.0,462.0,170.0,28.0,36.0


In [301]:
age_stat.columns

Index(['domain', '18-24', '25-34', '35-44', '45-54', '>=55'], dtype='object', name='age')

In [302]:
age_stat[['18-24', '25-34', '35-44', '45-54', '>=55']] = age_stat[['18-24', '25-34', '35-44', '45-54', '>=55']].div(age_stat[['18-24', '25-34', '35-44', '45-54', '>=55']].sum(axis=1), axis=0)
age_stat.head()

age,domain,18-24,25-34,35-44,45-54,>=55
0,b'zebra-zoya.ru',0.333333,0.000000,0.333333,0.000000,0.333333
1,b'news.yandex.ru',0.088496,0.391093,0.286041,0.159007,0.075364
2,b'sotovik.ru',0.071429,0.642857,0.142857,0.142857,0.000000
3,b'sweetrading.ru',0.000000,1.000000,0.000000,0.000000,0.000000
4,b'101.ru',0.085414,0.607096,0.223390,0.036794,0.047306


In [303]:
sex_age_stat = pd.merge(sex_stat, age_stat, on=['domain', 'domain'])
sex_age_stat.head()

,domain,F,M,18-24,25-34,35-44,45-54,>=55
0,b'zebra-zoya.ru',1.000000,0.000000,0.333333,0.000000,0.333333,0.000000,0.333333
1,b'news.yandex.ru',0.343134,0.656866,0.088496,0.391093,0.286041,0.159007,0.075364
2,b'sotovik.ru',0.250000,0.750000,0.071429,0.642857,0.142857,0.142857,0.000000
3,b'sweetrading.ru',0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
4,b'101.ru',0.395532,0.604468,0.085414,0.607096,0.223390,0.036794,0.047306


In [304]:
sex_age_stat.shape, len(sex_age_stat['domain'].unique())

((111581, 8), 111581)

Мы получили статистику по сайтам, которую можно будет применить к тестовой выборке при наличии соответствующего сайта в таблице sex_age_stat

Попробуем применить эту таблицу для train и test выборок и проверить результат

In [305]:
df5_train = pd.merge(df4_train, sex_age_stat, on=['domain', 'domain'], how='left')
for i in ['F', 'M', '18-24', '25-34', '35-44', '45-54', '>=55']:
    df5_train[i].fillna(df5_train[i].mean(), inplace=True)
df5_train.head()

,gender,age,uid,domain,day_part,season,week_day,day_type,F,M,18-24,25-34,35-44,45-54,>=55
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'zebra-zoya.ru',Afternoon,Winter,Saturday,Weekend,1.000000,0.000000,0.333333,0.000000,0.333333,0.000000,0.333333
1,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',Morning,Spring,Wednesday,Workday,0.343134,0.656866,0.088496,0.391093,0.286041,0.159007,0.075364
2,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',Morning,Spring,Wednesday,Workday,0.250000,0.750000,0.071429,0.642857,0.142857,0.142857,0.000000
3,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,b'sweetrading.ru',Night,Winter,Sunday,Weekend,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
4,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,b'101.ru',Night,Winter,Sunday,Weekend,0.395532,0.604468,0.085414,0.607096,0.223390,0.036794,0.047306


In [306]:
df5_test = pd.merge(df4_test, sex_age_stat, on=['domain', 'domain'], how='left')
for i in ['F', 'M', '18-24', '25-34', '35-44', '45-54', '>=55']:
    df5_test[i].fillna(df5_test[i].mean(), inplace=True)
df5_test.head()

,gender,age,uid,domain,day_part,season,week_day,day_type,F,M,18-24,25-34,35-44,45-54,>=55
0,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'interfax.ru',Afternoon,Winter,Sunday,Weekend,0.259488,0.740512,0.107679,0.410856,0.238305,0.161959,0.081200
1,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'amerikan-gruzovik.ru',Afternoon,Winter,Saturday,Weekend,0.392251,0.607749,0.101705,0.386701,0.306129,0.148989,0.056477
2,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'tv.jampo.tv',Morning,Winter,Saturday,Weekend,0.200000,0.800000,0.080000,0.426667,0.400000,0.013333,0.080000
3,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'sovsekretno.ru',Morning,Winter,Friday,Workday,0.190789,0.809211,0.026316,0.289474,0.414474,0.217105,0.052632
4,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'marketgid.com',Morning,Winter,Friday,Workday,0.348577,0.651423,0.054596,0.295847,0.228651,0.248717,0.172189


In [307]:
#Делаем весь столбец 'user_json' форматом json
#df['user_json'] = df['user_json'].apply(json.loads)
#result.set_index(['uid','timestamp']).url

#df.groupby(['uid']).count()

#dataset4.reset_index(drop = False, inplace = True)

#json_normalize(json.loads(df.iloc[0].user_json), 'visits')

#json_normalize(df.iloc[0].user_json, 'visits')

#df2_norm['domain'].astype(str)

#df2_norm.drop(labels=['timestamp','datetime'],axis=1, inplace=True)
#df2_norm.head()

#df2_drop[~df2_drop['url'].str.contains('http')]

#df2_drop.drop('domain', axis=1, inplace=True)
#part_stat = pd.DataFrame(df2_drop[['domain','gender']].groupby([pd.Grouper(key='domain'), 'part']).size(), \
#                        columns=['part_stat'])
#part_stat.head(10)
#df2_drop.drop(labels=['timestamp','datetime'],axis=1, inplace=True)

### Train test split, lable encoding

Когда вы очистили данные и сгенерировали признаки, которые можно дать на вход алгоритму, следующий этап — это разделить данные на тренировочную и тестовую выборки. Сохраните train и test выборки в отдельных файлах, используя метод `pandas.DataFrame.to_csv`. Либо просто сделайте два датафрейма: `train_df` и `test_df`. Обучите модель на ваш выбор, оцените результат, подумайте, как можно его улучшить.

Train:

In [53]:
df5_train['domain']   = le_domain  .transform(df5_train['domain'].astype(str))

In [308]:
#df5_train['domain']   = le_domain  .transform(df5_train['domain'].astype(str))
df5_train['day_part'] = le_day_part.transform(df5_train['day_part'].astype(str))
df5_train['season']   = le_season  .transform(df5_train['season'].astype(str))
df5_train['week_day'] = le_week_day.transform(df5_train['week_day'].astype(str))
df5_train['day_type'] = le_day_type.transform(df5_train['day_type'].astype(str))
df5_train['gender']   = le_gender  .transform(df5_train['gender'].astype(str))
df5_train['age']      = le_age     .transform(df5_train['age'].astype(str))

In [309]:
df5_train.head()

,gender,age,uid,domain,day_part,season,week_day,day_type,F,M,18-24,25-34,35-44,45-54,>=55
0,0,0,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'zebra-zoya.ru',0,3,2,0,1.000000,0.000000,0.333333,0.000000,0.333333,0.000000,0.333333
1,0,0,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'news.yandex.ru',2,1,6,1,0.343134,0.656866,0.088496,0.391093,0.286041,0.159007,0.075364
2,0,0,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,b'sotovik.ru',2,1,6,1,0.250000,0.750000,0.071429,0.642857,0.142857,0.142857,0.000000
3,1,1,d502331d-621e-4721-ada2-5d30b2c3801f,b'sweetrading.ru',3,3,3,0,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
4,1,1,d502331d-621e-4721-ada2-5d30b2c3801f,b'101.ru',3,3,3,0,0.395532,0.604468,0.085414,0.607096,0.223390,0.036794,0.047306


Test:

In [56]:
#df5_test['domain']   = le_domain  .transform(df5_test['domain'].astype(str))
df5_test['day_part'] = le_day_part.transform(df5_test['day_part'].astype(str))
df5_test['season']   = le_season  .transform(df5_test['season'].astype(str))
df5_test['week_day'] = le_week_day.transform(df5_test['week_day'].astype(str))
df5_test['day_type'] = le_day_type.transform(df5_test['day_type'].astype(str))

In [57]:
df5_test.head()

,gender,age,uid,domain,day_part,season,week_day,day_type,F,M,18-24,25-34,35-44,45-54,>=55
0,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'interfax.ru',0,3,3,0,0.259488,0.740512,0.107679,0.410856,0.238305,0.161959,0.081200
1,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'amerikan-gruzovik.ru',0,3,2,0,0.392251,0.607749,0.101705,0.386701,0.306129,0.148989,0.056477
2,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'tv.jampo.tv',2,3,2,0,0.200000,0.800000,0.080000,0.426667,0.400000,0.013333,0.080000
3,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'sovsekretno.ru',2,3,0,1,0.190789,0.809211,0.026316,0.289474,0.414474,0.217105,0.052632
4,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,b'marketgid.com',2,3,0,1,0.348577,0.651423,0.054596,0.295847,0.228651,0.248717,0.172189


Когда вы очистили данные и сгенерировали признаки, которые можно дать на вход алгоритму, следующий этап — это разделить данные на тренировочную и тестовую выборки. Сохраните train и test выборки в отдельных файлах, используя метод `pandas.DataFrame.to_csv`. Либо просто сделайте два датафрейма: `train_df` и `test_df`. Обучите модель на ваш выбор, оцените результат, подумайте, как можно его улучшить.

In [198]:
from sklearn.model_selection import train_test_split

In [390]:
x = np.array(df5_train.drop(labels=['gender','age','domain'],axis=1))
y = np.array(df5_train[['gender', 'age']])
x_test = np.array(df5_test.drop(labels=['gender','age','uid','domain'],axis=1))
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size = 0.33,random_state = 42)

In [391]:
x_train.shape, x_val.shape

((680176, 12), (335013, 12))

## Boosting

In [393]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import f1_score, accuracy_score, average_precision_score,\
                            confusion_matrix, roc_curve, precision_recall_curve, auc

import catboost as ctboost
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier as DT

In [394]:
#clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=4))

#ctb_gender = OneVsRestClassifier(ctboost.CatBoostClassifier(allow_writing_files=False))
#ctb_age = OneVsRestClassifier(XGBClassifier(n_jobs=4, max_depth=4))

Модель для гендера

In [395]:
ctb_gender = XGBClassifier(n_jobs=4, max_depth=4, n_estimators=10)
ctb_gender.fit(x_train[:,1:], y_train[:,0])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=4, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [205]:
#ctb_gender = XGBClassifier(n_jobs=4)

#parameters = {'max_depth': [2, 4, 10, 50, 100], 'n_estimators': [2, 4, 10, 20, 50, 100]}

#ctb_cv = (GridSearchCV(ctb_age, param_grid = parameters, scoring = 'accuracy', cv=5))

#ctb_cv.fit(x_train, y_train[:,0])
#print(ctb_cv.best_params_)

Результат по гендеру

In [396]:
predictions_gender = ctb_gender.predict(x_val[:,1:])
confusion = pd.DataFrame(confusion_matrix(y_val[:,0], predictions_gender))

TN, FP = confusion[0][0], confusion[0][1]
FN, TP = confusion[1][0], confusion[1][1]

print('Confusion matrix for Decision Tree:')
data = np.array([['','Predicted Zeros','Predicted Ones'],
            ['Zeros',TN,FP],
            ['Ones',FN,TP]])

print(pd.DataFrame(data=data[1:,1:],
              index=data[1:,0],
              columns=data[0,1:]))

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Confusion matrix for Decision Tree:
      Predicted Zeros Predicted Ones
Zeros           73813          29191
Ones            64894         167115


In [397]:
ctb_gender.score(x_val[:,1:],y_val[:,0])

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7191601519941018

Модель для возраста

In [208]:
#mlb = MultiLabelBinarizer()

#y_age = mlb.fit_transform(y_train[:,1].reshape(-1,1))

#y_val_t = mlb.transform(y_val[:,1].reshape(-1,1))

In [209]:
#ctb_age = XGBClassifier(n_jobs=4)

#parameters = {'max_depth': [2, 4, 10, 50, 100], 'n_estimators': [2, 4, 10, 20, 50, 100]}

#ctb_cv = OneVsRestClassifier((GridSearchCV(ctb_age, param_grid = parameters, scoring = 'accuracy', cv=5)))

#ctb_cv.fit(x_train, y_train[:,1])
#print(ctb_cv.best_params_)

In [398]:
ctb_age = OneVsOneClassifier(XGBClassifier(n_jobs=4, max_depth=10, n_estimators=10))
ctb_age.fit(x_train[:,1:], y_train[:,1])

OneVsOneClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=4, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          n_jobs=1)

Результат по возрасту

In [211]:
predictions_age = ctb_age.predict(x_val[:,1:])
confusion = pd.DataFrame(confusion_matrix(y_val[:,1], predictions_age))

pd.DataFrame(confusion)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

,0,1,2,3,4
0,5844,17479,7257,519,173
1,1076,83847,35846,2142,802
2,770,36704,69175,2304,907
3,348,19497,20816,10329,874
4,136,7956,6287,727,3198


In [212]:
ctb_age.score(x_val[:,1:],y_val[:,1])

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

0.5145860011402542

Overall score:

In [213]:
res_pd = pd.DataFrame(y_val) == pd.DataFrame(np.hstack((predictions_gender.reshape(-1,1),predictions_age.reshape(-1,1))))
res_pd[((res_pd[0] == True) & (res_pd[1] == True))].shape[0]/y_val.shape[0]

0.39799052574079213

In [399]:
decision_tree = DT(criterion='gini', min_samples_leaf=100, max_depth=10)

In [400]:
decision_tree.fit(x_train[:,1:],y_train[:,1])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=100, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [401]:
predictions_age = decision_tree.predict(x_val[:,1:])
confusion = pd.DataFrame(confusion_matrix(y_val[:,1], predictions_age))

pd.DataFrame(confusion)

,0,1,2,3,4
0,7173,16888,6681,365,165
1,2506,85665,32969,1867,706
2,1783,38095,67141,1772,1069
3,832,19310,20964,9738,1020
4,196,7771,6616,550,3171


In [402]:
decision_tree.score(x_val[:,1:],y_val[:,1])

0.516063555742613

In [403]:
uid = x_val[:,0].reshape(-1,1)

In [404]:
gender = ctb_gender.predict_proba(x_val[:,1:])[:,1].reshape(-1,1)

In [406]:
age = predictions_age.reshape(-1,1)

In [407]:
gender_y = y_val[:,0].reshape(-1,1)

In [408]:
age_y = y_val[:,1].reshape(-1,1)

In [409]:
uid.shape, gender.shape, age.shape, y_val.shape

((335013, 1), (335013, 1), (335013, 1), (335013, 2), (335013, 1))

In [410]:
result = pd.DataFrame(np.hstack([uid,gender,age,gender_y,age_y]), columns=['uid', 'gender','age', 'gender_y','age_y'])

#result['gender'] = result['gender'].astype(int)
#result['age'] = result['age'].astype(int)
#result['gender_y'] = result['gender_y'].astype(int)
#result['age_y'] = result['age_y'].astype(int)
result['uid'].unique().shape, result.shape

((30252,), (335013, 5))

In [373]:
result.head()

,uid,gender,age,gender_y,age_y
0,c1327dcd-9fb7-44c7-821c-68d926bba019,0.627931,2,1,2
1,2438f09a-437a-434b-9f57-607d2736741a,0.507998,2,1,4
2,6948734e-e70c-4c59-acd3-9e04d08b3adf,0.51273,2,0,0
3,c65b96cb-c316-4818-84bc-896788db3a11,0.556249,1,0,2
4,6568a249-e4cf-4630-b9c5-a2660ac0c32e,0.618214,1,0,0


In [414]:
result_fin = result_int.groupby('uid').agg(lambda x:x.value_counts().index[0]).reset_index(drop=False)
print(result_fin.shape)

(30252, 4)


In [374]:
result['distance']=abs(result['gender']-0.5)

result_sort = result.sort_values(by='distance',axis = 0, ascending = True).reset_index(drop=True)

result_sort['gender'].loc[:round((result_sort.shape[0]/2)-2)] = '-'
result_sort['gender'].loc[round((result_sort.shape[0]/2)-2)+1:] = (round(result_sort['gender'].loc[round((result_sort.shape[0]/2)-2)+1:].astype(float))).astype(int)
result_sort.drop(labels=['distance'],axis=1, inplace=True)

result_sort.head()

,uid,gender,age,gender_y,age_y
0,4256f85e-222b-46de-8195-824cd4f5c755,-,1,0,1
1,558fb978-7da8-4bb2-bbf5-3f9b14a9aa16,-,2,1,2
2,292ac99d-69d0-481e-9bbb-c5d0f6b158c4,-,2,0,3
3,d25d414c-6cfc-4e3e-9466-52cfddbdd375,-,2,1,2
4,af61fde6-dabd-417a-b252-c93c11c6dfa6,-,1,1,2


In [282]:
#result['gender'] = result['gender'].map(lambda item : '-' if item < 0.55 and item > 0.45 else round(item))
#result['gender'].value_counts()['-']/(result['gender'].value_counts()['M']+result['gender'].value_counts()['F'])

In [290]:
result['gender'] = result['gender'].replace(0, 'F')
result['gender'] = result['gender'].replace(1, 'M')

### Сохранение модели

Обучать модель вы можете в ноутбуке - это удобно. А после того как модель обучилась и стала выдавать приемлемое качество на вашей валидационной выборке, сохраните ее в отдельном файле (например, используя pickle):

In [318]:
#Сохранить модель, которая содержится в переменной vectorizer
#Установите правильные атрибуты файла модели, чтобы он мог быть считан проверочным скриптом:
model_file_1 = "project01_model_gender.pickle"
model_file_2 = "project01_model_age.pickle"

with open(os.environ['HOME'] + '/' + model_file_1, 'wb') as f:
    pickle.dump(ctb_gender, f)
    
with open(os.environ['HOME'] + '/' + model_file_2, 'wb') as f:
    pickle.dump(decision_tree, f) 
    
os.chmod(os.environ['HOME'] + '/' + model_file_1, 0o644)
os.chmod(os.environ['HOME'] + '/' + model_file_2, 0o644)

Сохраняем также Label encoding модели:

In [343]:
m_le_day_part = "le_day_part.pickle"
m_le_season   = "le_season  .pickle"
m_le_week_day = "le_week_day.pickle"
m_le_day_type = "le_day_type.pickle"
m_le_gender   = "le_gender  .pickle"
m_le_age      = "le_age     .pickle"

with open(os.environ['HOME'] + '/' + m_le_day_part, 'wb') as f:
    pickle.dump(le_day_part, f)
with open(os.environ['HOME'] + '/' + m_le_season, 'wb') as f:
    pickle.dump(le_season, f)
with open(os.environ['HOME'] + '/' + m_le_week_day, 'wb') as f:
    pickle.dump(le_week_day, f)
with open(os.environ['HOME'] + '/' + m_le_day_type, 'wb') as f:
    pickle.dump(le_day_type, f)
with open(os.environ['HOME'] + '/' + m_le_gender, 'wb') as f:
    pickle.dump(le_gender, f)
with open(os.environ['HOME'] + '/' + m_le_age, 'wb') as f:
    pickle.dump(le_age, f)

### Сохранение таблицы со статистикой

In [90]:
sex_age_stat.head()

,domain,F,M,18-24,25-34,35-44,45-54,>=55
0,b'zebra-zoya.ru',1.000000,0.000000,0.333333,0.000000,0.333333,0.000000,0.333333
1,b'news.yandex.ru',0.343134,0.656866,0.088496,0.391093,0.286041,0.159007,0.075364
2,b'sotovik.ru',0.250000,0.750000,0.071429,0.642857,0.142857,0.142857,0.000000
3,b'sweetrading.ru',0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
4,b'101.ru',0.395532,0.604468,0.085414,0.607096,0.223390,0.036794,0.047306


In [321]:
sex_age_stat.to_csv('sex_age_stat.csv',sep='\t', index=False)

Однако, не забудьте сохранить и код генерации признаков и обучения модели - это нужно для воспроизводимости результатов.

### Обработка тестовых данных и формат вывода результатов

Помимо того, что у вас должна получиться точная модель, вам нужно уложиться в SLA (service-level agreement). Всё почти как по-настоящему. Результатом вашей работы в данном случае будет не выходной файл, в котором вы всё посчитали для скрытой выборки, а скрипт, который мы будем запускать и проверять SLA и точность.

Создайте в корне своей домашней директории файл `project01_gender-age.py`. 

Назначте ему нужные права: `chmod 755 project01_gender-age.py`.

Вот фрагмент кода, который считывает данные из потока стандартного ввода:

In [19]:
#!/opt/anaconda/envs/bd9/bin/python

import sys
import pandas as pd

columns=['gender','age','uid','user_json']

df = pd.read_table(
    sys.stdin, 
    sep='\t', 
    header=None, 
    names=columns
)

Далее вы должны применить все те же самые преобразования данных, которые (возможно) применили к обучающей выборке.
Затем считываете модель:

In [ ]:
#считать модель из файла в переменную vectorizer
import pickle

model_file = "project01_model.pickle"
p1 = pickle.load(open(model_file, 'rb'))

Путь к модели необходимо указывать относительно вашей домашней директории. Например, если в директории `name.surname` есть папка `project01`, в которой лежит модель `project01_model.pickle`, то путь к модели в скрипте будет выглядеть как `project01/project01_model.pickle`.

Задача вашего скрипта сделать предсказание по всем полученным строкам и выдать результат в формате json. В файле должны быть только те пользователи, у которых пол и возрастная категория изначально неизвестны, и они должны быть **отсортированы по UID по возрастанию значений лексикографически.** Пример вывода указан ниже.

In [ ]:
output = output[['uid', 'gender', 'age']]
output.sort_values(by='uid',axis = 0, ascending = True, inplace = True)
sys.stdout.write(output.to_json(orient='records'))

Для самопроверки вы можете локально оттестировать свой скрипт, используя следующую команду:

In [ ]:
!tail -n1000 gender_age_dataset.txt | ./project01_gender-age.py > output.json

### Подсказки

1. Есть много различных способов решить данную задачу: можно просто хорошо поработать с урлами и доменами, можно пропарсить содержимое этих урлов (заголовки, текст и т.д.) и воспользоваться неким векторизатором типа TF\*IDF для генерации дополнительных фич, которые уже в дальнейшем вы подадите на вход ML-алгоритму, можно сделать тематическое моделирование (LDA, BigARTM) сайтов и использовать одну или несколько тем в качестве фич.

2. Возможно, что данные грязные и их нужно дополнительно обработать. Спецсимволы, кириллические домены? Уделите этому этапу достаточно времени: здесь чистота датасета важнее, чем выбор алгоритма.

3. Часто бывает, что лучшее решение с точки зрения результата — оно же самое простое. Попробуйте сначала простые способы, простые алгоритмы, прежде чем переходить к тяжёлой артиллерии. Один из вариантов — начать с небольшого RandomForest.

4. Вам почти наверняка понадобится что-то из пакета sklearn. [Документация](http://scikit-learn.org/stable/user_guide.html) — ваш лучший друг.

5. Вы можете сначала предсказать пол, а затем возраст, либо сразу и то, и другое. Экспериментируйте.

6. Объединяйтесь в команды. Так гораздо веселее и интереснее.

7. Ведите дневник того, что вы делаете: подход, его основная идея, использованные фичи, точность предсказания и т.д. Это хорошая практика, позволяющая не забыть то, что вы делали до этого и позволяющая получить знание о том, что вам не принесло результата. Эту практику продемонстрировал участник одного из наших прошлых запусков – Сергей Зайцев. Мы решили ее добавить в раздел Подсказки.

### Проверка
Проверка осуществляется из [Личного кабинета](http://lk.newprolab.com/lab/project1). До дедлайна вы будете проверять работу своего скрипта на валидационной выборке (2 000 записей). При наступлении дедлайна мы автоматически пересчитаем модели по скрытой тестовой выборке (3 000 записей). Это и будет финальным результатом.

* В поле `part of users with predicted gender + age` - указана доля пользователей, которая была предсказана от общего числа неизвестных пользователей (пример: по 3 000 был сделан прогноз, а всего было неизвестно 5 000, чекер выдаст 0.6).

* В поле `correctly predicted users / total number of users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа всех пользователей (пример: по 3 000 был сделан прогноз, правильно было спрогнозировано 1 500, а всего было неизвестно 5 000, чекер выдаст 0.3)

* В поле `correctly predicted users / number of predicted users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа предсказанных пользователей (пример: по 3 000 был сделан прогноз, из них правильно предсказано 1 500, чекер выдаст 0.5).

**Важное замечание!** Вы должны дать прогноз хотя бы по 50% пользователей, у которых изначально не указан пол и возрастная категория. Иными словами, вы можете оставить неопределенными не более 50% изначально неопределенных пользователей.

**Если доля в последнем поле превысит порог 0.28, то проект будет засчитан, при условии что выполнен SLA в 0.04 секунды на каждого пользователя (т.е. на каждую строчку тестового датасета)**

Лучшей команде, набравшей максимальный результат, мы подарим специальный приз, о котором скажем позднее.